In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
sc.stop()

spark = (
    SparkSession.builder
    .appName("Desafio2")
    .getOrCreate()
)

In [2]:
!aws s3 cp s3://722687989925-raw/ ./dadosDaRaw --recursive

download: s3://722687989925-raw/multas.csv to dadosDaRaw/multas.csv


In [ ]:
df = spark.read.csv("./dadosDaRaw/multas.csv", header=True)

In [ ]:
#casting columns
from  pyspark.sql.functions import to_date
from pyspark.sql.functions import col

df_transform = df.withColumn(
    "ultima_multa",
    to_date("ultima_multa", "yyyy-MM-dd")
)
df_transform = df_transform.withColumn(
    'total_multas', col('total_multas').cast('int'))

In [ ]:
#dropping rows where sharing data was not allowed
df_transform = df_transform.where(df_transform.compartilhou_dados == 'Sim')
df_transform = df_transform.drop("compartilhou_dados")

In [31]:
#particionando por estado
df_transform.write.partitionBy('estado').mode('overwrite').parquet('trusted/')

In [4]:
#uploading to trusted
!aws s3 cp ./trusted/ s3://722687989925-trusted/ --recursive


upload: trusted/estado=AC/.part-00000-12613c9b-b5e7-413b-81a5-233e587f1d73.c000.snappy.parquet.crc to s3://722687989925-trusted/estado=AC/.part-00000-12613c9b-b5e7-413b-81a5-233e587f1d73.c000.snappy.parquet.crc
upload: trusted/estado=AP/.part-00000-12613c9b-b5e7-413b-81a5-233e587f1d73.c000.snappy.parquet.crc to s3://722687989925-trusted/estado=AP/.part-00000-12613c9b-b5e7-413b-81a5-233e587f1d73.c000.snappy.parquet.crc
upload: trusted/estado=AM/.part-00000-12613c9b-b5e7-413b-81a5-233e587f1d73.c000.snappy.parquet.crc to s3://722687989925-trusted/estado=AM/.part-00000-12613c9b-b5e7-413b-81a5-233e587f1d73.c000.snappy.parquet.crc
upload: trusted/estado=AP/part-00000-12613c9b-b5e7-413b-81a5-233e587f1d73.c000.snappy.parquet to s3://722687989925-trusted/estado=AP/part-00000-12613c9b-b5e7-413b-81a5-233e587f1d73.c000.snappy.parquet
upload: trusted/estado=AL/part-00000-12613c9b-b5e7-413b-81a5-233e587f1d73.c000.snappy.parquet to s3://722687989925-trusted/estado=AL/part-00000-12613c9b-b5e7-413b-81a

NameError: name 'df_transform' is not defined